In [42]:
from skimage import io
import PIL.Image
import numpy as np
import os

In [43]:
#Feature Matrix
array_of_samples = np.ones((1,43200))
directory = os.fsencode('RGB_PNG')
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".png"):   #ignore files ending with .ipynb_checkpoints
        img = io.imread('RGB_PNG/' + filename) #load image
        four_dim_array = np.array(img)
        one_dim_array = four_dim_array.reshape(1,43200) #convert into one long array
        array_of_samples = np.vstack([array_of_samples, one_dim_array])
print(array_of_samples.shape)
array_of_samples = array_of_samples[1:][:] #delete the first row of ones which was used only to initialize the array
print(array_of_samples.shape)

(722, 43200)
,(721, 43200)


In [44]:
#Labels
import pandas as pd
my_csv = pd.read_csv('pokemon.csv')
my_csv = my_csv[:721]
my_csv = my_csv.sort_values(by = 'Name')      
labels = np.array(my_csv.Type1)


#Conversion of types(labels) to numbers
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labels = le.fit_transform(labels)
labels = np.expand_dims(labels, axis = 1) #conversion into a correct vector
labels.shape

(721, 1)

In [45]:
#Split into Training and Test sets

array_of_samples /= 255

from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(array_of_samples, labels, test_size=0.20, random_state=21, stratify = labels)
x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

y_train = y_train.flatten()
y_valid = y_valid.flatten()                  

x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((576, 43200), (145, 43200), (576,), (145,))

In [46]:
#Categorize Labels
import tensorflow.keras as keras
num_classes = 18               #18 pokemon types

y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

In [52]:
#Build model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units = 512, activation='relu', input_shape=(43200,)))
model.add(Dense(units = 512, activation='relu'))
model.add(Dense(units = num_classes, activation='softmax'))

In [53]:
model.summary()

Model: "sequential_5"
,_________________________________________________________________
, Layer (type)                Output Shape              Param #   
,=================================================================
, dense_15 (Dense)            (None, 512)               22118912  
,                                                                 
, dense_16 (Dense)            (None, 512)               262656    
,                                                                 
, dense_17 (Dense)            (None, 18)                9234      
,                                                                 
,=================================================================
,Total params: 22,390,802
,Trainable params: 22,390,802
,Non-trainable params: 0
,_________________________________________________________________


In [54]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

In [55]:
model.fit(x_train, y_train, epochs=20, verbose=1, validation_data=(x_valid, y_valid))


Epoch 1/20
,18/18 [==============================] - 6s 306ms/step - loss: 9.7596 - accuracy: 0.1059 - val_loss: 3.2137 - val_accuracy: 0.0966
,Epoch 2/20
,18/18 [==============================] - 5s 279ms/step - loss: 2.8399 - accuracy: 0.1424 - val_loss: 2.8822 - val_accuracy: 0.1103
,Epoch 3/20
,18/18 [==============================] - 5s 275ms/step - loss: 2.4896 - accuracy: 0.2378 - val_loss: 3.1588 - val_accuracy: 0.1517
,Epoch 4/20
,18/18 [==============================] - 5s 289ms/step - loss: 2.2674 - accuracy: 0.2934 - val_loss: 2.9017 - val_accuracy: 0.1448
,Epoch 5/20
,18/18 [==============================] - 5s 307ms/step - loss: 1.9943 - accuracy: 0.3837 - val_loss: 3.3223 - val_accuracy: 0.1655
,Epoch 6/20
,18/18 [==============================] - 7s 373ms/step - loss: 1.7491 - accuracy: 0.4514 - val_loss: 3.1231 - val_accuracy: 0.1655
,Epoch 7/20
,18/18 [==============================] - 8s 436ms/step - loss: 1.4820 - accuracy: 0.5278 - val_loss: 3.4283 - val_accuracy: 